In [2]:
"""
Analyze OnlineRetail.csv to answer:
1) Which products are our bestsellers?
2) When do customers buy most?

Requirements:
- pip install pandas matplotlib
"""

import pandas as pd

# ----------- CONFIG -----------
DATA_PATH = "OnlineRetail.csv"   # change to full path if needed
TOP_N_PRODUCTS = 10
# ------------------------------


def load_and_clean_data(path: str) -> pd.DataFrame:
    """Load OnlineRetail.csv and apply basic cleaning."""
    # encoding="ISO-8859-1" works for this dataset
    df = pd.read_csv(path, encoding="ISO-8859-1")

    # Parse dates
    df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

    # Remove cancelled orders: InvoiceNo starting with 'C'
    df = df[~df["InvoiceNo"].astype(str).str.startswith("C")]

    # Remove negative or zero quantities and prices
    df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]

    # Drop rows with missing product description
    df = df.dropna(subset=["Description"])

    # Compute total sales value
    df["TotalSales"] = df["Quantity"] * df["UnitPrice"]

    return df


def analyze_bestsellers(df: pd.DataFrame, top_n: int = 10) -> pd.DataFrame:
    """
    Find bestselling products:
    - By total quantity sold
    - Also show total revenue and number of invoices
    """
    product_group = (
        df.groupby("Description")
          .agg(
              TotalQuantity=("Quantity", "sum"),
              TotalRevenue=("TotalSales", "sum"),
              NumInvoices=("InvoiceNo", "nunique"),
          )
          .sort_values(by="TotalQuantity", ascending=False)
    )

    top_products = product_group.head(top_n)
    return top_products


def analyze_purchase_timing(df: pd.DataFrame) -> dict:
    """
    Analyze when customers buy most:
    - By hour of day
    - By day of week
    - By month
    """
    # Time-based features
    df["Hour"] = df["InvoiceDate"].dt.hour
    df["DayOfWeek"] = df["InvoiceDate"].dt.day_name()
    df["Month"] = df["InvoiceDate"].dt.to_period("M").astype(str)

    # Aggregate by quantity (you could also use TotalSales)
    by_hour = (
        df.groupby("Hour")["Quantity"]
        .sum()
        .sort_values(ascending=False)
    )
    by_dow = (
        df.groupby("DayOfWeek")["Quantity"]
        .sum()
        .sort_values(ascending=False)
    )
    by_month = (
        df.groupby("Month")["Quantity"]
        .sum()
        .sort_values(ascending=False)
    )

    return {
        "by_hour": by_hour,
        "by_dow": by_dow,
        "by_month": by_month,
    }


def main():
    print("Loading and cleaning data...")
    df = load_and_clean_data(DATA_PATH)

    # 1) Bestselling products
    print("\n=== Top Products (Bestsellers by Quantity) ===")
    top_products = analyze_bestsellers(df, top_n=TOP_N_PRODUCTS)
    print(top_products)

    # 2) When do customers buy most?
    timing_stats = analyze_purchase_timing(df)

    print("\n=== When Do Customers Buy Most? (by Hour of Day) ===")
    print(timing_stats["by_hour"])
    best_hour = timing_stats["by_hour"].idxmax()
    print(f"\nPeak hour (by quantity): {best_hour}:00")

    print("\n=== When Do Customers Buy Most? (by Day of Week) ===")
    print(timing_stats["by_dow"])
    best_dow = timing_stats["by_dow"].idxmax()
    print(f"\nPeak day of week (by quantity): {best_dow}")

    print("\n=== When Do Customers Buy Most? (by Month) ===")
    print(timing_stats["by_month"])
    best_month = timing_stats["by_month"].idxmax()
    print(f"\nPeak month (by quantity): {best_month}")

    # If you want, you could also save outputs:
    # top_products.to_csv("bestselling_products.csv")
    # timing_stats["by_hour"].to_csv("sales_by_hour.csv")
    # timing_stats["by_dow"].to_csv("sales_by_day_of_week.csv")
    # timing_stats["by_month"].to_csv("sales_by_month.csv")


if __name__ == "__main__":
    main()


Loading and cleaning data...

=== Top Products (Bestsellers by Quantity) ===
                                    TotalQuantity  TotalRevenue  NumInvoices
Description                                                                 
PAPER CRAFT , LITTLE BIRDIE                 80995     168469.60            1
MEDIUM CERAMIC TOP STORAGE JAR              78033      81700.92          247
WORLD WAR 2 GLIDERS ASSTD DESIGNS           55047      13841.85          535
JUMBO BAG RED RETROSPOT                     48474      94340.05         2089
WHITE HANGING HEART T-LIGHT HOLDER          37891     106292.77         2256
POPCORN HOLDER                              36761      34298.87          803
ASSORTED COLOUR BIRD ORNAMENT               36461      59094.93         1455
PACK OF 72 RETROSPOT CAKE CASES             36419      21259.10         1320
RABBIT NIGHT LIGHT                          30788      66964.99          994
MINI PAINT SET VINTAGE                      26633      16937.82          380